In [1]:
import tensorflow as tf 
import numpy as np 
from tensorflow import keras 
import os 
import math 
import random 
import pickle 
import matplotlib.pyplot as plt 
from collections import deque 

from vehicle_model_DDQN3_2 import Environment 
from cell_model import CellModel 

os.environ['CUDA_VISIBLE_DEVICES'] = '-1'

In [2]:
drving_cycle = '../../OC_SIM_DB/OC_SIM_DB_Cycles/Highway/01_FTP72_fuds.mat'
battery_path = "../../OC_SIM_DB/OC_SIM_DB_Bat/OC_SIM_DB_Bat_nimh_6_240_panasonic_MY01_Prius.mat"
motor_path = "../../OC_SIM_DB/OC_SIM_DB_Mot/OC_SIM_DB_Mot_pm_95_145_X2.mat"
cell_model = CellModel()
env = Environment(cell_model, drving_cycle, battery_path, motor_path, 1)


In [3]:
# STATE_SIZE = env.calculation_comp["state_size"]
STATE_SIZE = 4
ACTION_SIZE = env.calculation_comp["action_size"] 
LEARNING_RATE = 0.00025 

TOTAL_EPISODES = 200
MAX_STEPS = 50000 

GAMMA = 0.95 

MAX_EPSILON = 1 
MIN_EPSILON = 0.01 
DECAY_RATE = 0.00002
BATCH_SIZE = 32 
TAU = 0.001 
DELAY_TRAINING = 3000 
EPSILON_MIN_ITER = 5000

In [4]:
primary_network = keras.Sequential([
    keras.layers.Dense(30, activation="relu", kernel_initializer=keras.initializers.he_normal()), 
#     keras.layers.BatchNormalization(), 
    keras.layers.Dense(30, activation="relu", kernel_initializer=keras.initializers.he_normal()),
#     keras.layers.BatchNormalization(), 
    keras.layers.Dense(ACTION_SIZE),
])
target_network = keras.Sequential([
    keras.layers.Dense(30, activation="relu", kernel_initializer=keras.initializers.he_normal()), 
#     keras.layers.BatchNormalization(), 
    keras.layers.Dense(30, activation="relu", kernel_initializer=keras.initializers.he_normal()),
#     keras.layers.BatchNormalization(), 
    keras.layers.Dense(ACTION_SIZE),
])

primary_network.compile(
    loss="mse", 
    optimizer=keras.optimizers.Adam(lr=LEARNING_RATE) 
)

# for t, p in zip(target_network.trainable_variables, primary_network.trainable_variables): 
#     t.assign(p)

In [5]:
def update_network(primary_network, target_network): 
    for t, p in zip(target_network.trainable_variables, primary_network.trainable_variables): 
        t.assign(t * (1 - TAU) + p * TAU)

In [6]:
class Memory: 
    def __init__(self, max_memory): 
        self.max_memory = max_memory 
        self._samples = [] 
        
    def add_sample(self, sample): 
        self._samples.append(sample)
        if len(self._samples) > self.max_memory: 
            self._samples.pop(0)
        
    def sample(self, no_samples): 
        if no_samples > len(self._samples): 
            return random.sample(self._samples, len(self._samples))
        else: 
            return random.sample(self._samples, no_samples)
    
    @property
    def num_samples(self):
        return len(self._samples)
    

# memory = Memory(10000)

In [7]:
def choose_action(state, primary_network, eps): 
    if random.random() < eps: 
        return random.randint(0, ACTION_SIZE - 1)
    else: 
        return np.argmax(primary_network(np.array(state).reshape(1, -1))) 

In [8]:
def train(primary_network, target_network, memory): 
    batch = memory.sample(BATCH_SIZE)
    states = np.array([val[0] for val in batch]) 
    actions = np.array([val[1] for val in batch])
    rewards = np.array([val[2] for val in batch])
    next_states = np.array([np.zeros(STATE_SIZE) if val[3] is None else val[3]  
                            for val in batch])
    
    prim_qt = primary_network(states)
    prim_qtp1 = primary_network(next_states)
    target_q = prim_qt.numpy() 
    updates = rewards 
    valid_idxs = next_states.sum(axis=1) != 0 
    batch_idxs = np.arange(BATCH_SIZE)
    prim_action_tp1 = np.argmax(prim_qtp1.numpy(), axis=1)
    q_from_target = target_network(next_states)
    updates[valid_idxs] += GAMMA * q_from_target.numpy()[batch_idxs[valid_idxs], 
                                                        prim_action_tp1[valid_idxs]]
    
    target_q[batch_idxs, actions] = updates 
    loss = primary_network.train_on_batch(states, target_q)
    return loss 
    
    
    

In [9]:
def initialization_with_rewardFactor(reward_factor):
    env = Environment(cell_model, drving_cycle, battery_path, motor_path, reward_factor)
    
    memory = Memory(10000)
    
    primary_network = keras.Sequential([
        keras.layers.Dense(30, activation="relu", kernel_initializer=keras.initializers.he_normal()),
#         keras.layers.BatchNormalization(),  
        keras.layers.Dense(30, activation="relu", kernel_initializer=keras.initializers.he_normal()),
#         keras.layers.BatchNormalization(), 
        keras.layers.Dense(ACTION_SIZE),
    ])
    target_network = keras.Sequential([
        keras.layers.Dense(30, activation="relu", kernel_initializer=keras.initializers.he_normal()), 
#         keras.layers.BatchNormalization(), 
        keras.layers.Dense(30, activation="relu", kernel_initializer=keras.initializers.he_normal()),
#         keras.layers.BatchNormalization(), 
        keras.layers.Dense(ACTION_SIZE),
    ])
    primary_network.compile(
        loss="mse", 
        optimizer=keras.optimizers.Adam(lr=LEARNING_RATE) 
    )
    return env, memory, primary_network, target_network 
    

In [10]:
print("environment version: {}".format(env.version)) 

 
reward_factors = [10]
results_dict = {} 

for reward_factor in reward_factors: 
    eps = MAX_EPSILON 
    steps = 0
    episode_rewards = [] 
    episode_SOCs = [] 
    episode_FCs = [] 
    
    env, memory, primary_network, target_network = initialization_with_rewardFactor(reward_factor)
    for episode in range(TOTAL_EPISODES): 
        state = env.reset() 
        avg_loss = 0 
        total_reward = 0
        cnt = 1 

        while True:
            action = choose_action(state, primary_network, eps)
            next_state, reward, done = env.step(action)
            total_reward += reward 
            if done: 
                next_state = None 
            memory.add_sample((state, action, reward, next_state))

            if steps > DELAY_TRAINING: 
                loss = train(primary_network, target_network, memory)
                update_network(primary_network, target_network)
                eps = MIN_EPSILON + (MAX_EPSILON - MIN_EPSILON) * np.exp(-DECAY_RATE * steps)
            else: 
                loss = -1

            avg_loss += loss 
            steps += 1 

            if done: 
                if steps > DELAY_TRAINING: 
                    SOC_deviation_history = np.sum(np.abs(np.array(env.history["SOC"]) - 0.6)) 
                    avg_loss /= cnt 
                    print('Episode: {}'.format(episode + 1),
                          'Total reward: {}'.format(total_reward), 
                          'Explore P: {:.4f}'.format(eps), 
                          "SOC: {:.4f}".format(env.SOC), 
                         "Cumulative_SOC_deviation: {:.4f}".format(SOC_deviation_history), 
                         "Fuel Consumption: {:.4f}".format(env.fuel_consumption), 
                         )
                else: 
                    print(f"Pre-training...Episode: {episode}")
                
                episode_rewards.append(total_reward)
                episode_SOCs.append(env.SOC)
                episode_FCs.append(env.fuel_consumption)
                break 

            state = next_state 
            cnt += 1 
    
    results_dict[reward_factor] = {
        "rewards": episode_rewards, 
        "SOCs": episode_SOCs, 
        "FCs": episode_FCs 
    }
            
    

environment version: 2
maximum steps, simulation is done ... 
Pre-training...Episode: 0
maximum steps, simulation is done ... 
Pre-training...Episode: 1

If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.


If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer

maximum steps, simulation is done ... 
Episode: 41 Total reward: -3884.7399151107034 Explore P: 0.3311 SOC: 1.0000 Cumulative_SOC_deviation: 377.8473 Fuel Consumption: 106.2666
maximum steps, simulation is done ... 
Episode: 42 Total reward: -3697.919689856714 Explore P: 0.3224 SOC: 1.0000 Cumulative_SOC_deviation: 359.8212 Fuel Consumption: 99.7075
maximum steps, simulation is done ... 
Episode: 43 Total reward: -2397.1549629288625 Explore P: 0.3140 SOC: 1.0000 Cumulative_SOC_deviation: 231.1431 Fuel Consumption: 85.7243
maximum steps, simulation is done ... 
Episode: 44 Total reward: -3442.062397175935 Explore P: 0.3057 SOC: 1.0000 Cumulative_SOC_deviation: 334.9056 Fuel Consumption: 93.0067
maximum steps, simulation is done ... 
Episode: 45 Total reward: -3213.709397988764 Explore P: 0.2977 SOC: 1.0000 Cumulative_SOC_deviation: 311.4280 Fuel Consumption: 99.4289
maximum steps, simulation is done ... 
Episode: 46 Total reward: -2182.9807093687155 Explore P: 0.2899 SOC: 1.0000 Cumulat

maximum steps, simulation is done ... 
Episode: 88 Total reward: -371.9204843542257 Explore P: 0.0983 SOC: 0.6632 Cumulative_SOC_deviation: 31.8127 Fuel Consumption: 53.7940
maximum steps, simulation is done ... 
Episode: 89 Total reward: -405.3408538660224 Explore P: 0.0960 SOC: 0.6734 Cumulative_SOC_deviation: 35.0643 Fuel Consumption: 54.6982
maximum steps, simulation is done ... 
Episode: 90 Total reward: -480.5434038728673 Explore P: 0.0936 SOC: 0.6962 Cumulative_SOC_deviation: 42.4306 Fuel Consumption: 56.2377
maximum steps, simulation is done ... 
Episode: 91 Total reward: -473.2503489973189 Explore P: 0.0914 SOC: 0.7078 Cumulative_SOC_deviation: 41.5722 Fuel Consumption: 57.5280
maximum steps, simulation is done ... 
Episode: 92 Total reward: -353.26379006508637 Explore P: 0.0892 SOC: 0.6572 Cumulative_SOC_deviation: 29.9504 Fuel Consumption: 53.7597
maximum steps, simulation is done ... 
Episode: 93 Total reward: -329.3110790849326 Explore P: 0.0870 SOC: 0.6926 Cumulative_SOC_

maximum steps, simulation is done ... 
Episode: 135 Total reward: -282.3312363077049 Explore P: 0.0343 SOC: 0.6048 Cumulative_SOC_deviation: 23.2330 Fuel Consumption: 50.0012
maximum steps, simulation is done ... 
Episode: 136 Total reward: -282.951724850679 Explore P: 0.0336 SOC: 0.6102 Cumulative_SOC_deviation: 23.2525 Fuel Consumption: 50.4264
maximum steps, simulation is done ... 
Episode: 137 Total reward: -395.92145845879026 Explore P: 0.0330 SOC: 0.6139 Cumulative_SOC_deviation: 34.5402 Fuel Consumption: 50.5193
maximum steps, simulation is done ... 
Episode: 138 Total reward: -315.16784649962705 Explore P: 0.0324 SOC: 0.6083 Cumulative_SOC_deviation: 26.6374 Fuel Consumption: 48.7940
maximum steps, simulation is done ... 
Episode: 139 Total reward: -315.14856171116406 Explore P: 0.0318 SOC: 0.6166 Cumulative_SOC_deviation: 26.5641 Fuel Consumption: 49.5076
maximum steps, simulation is done ... 
Episode: 140 Total reward: -294.78137227212596 Explore P: 0.0312 SOC: 0.5987 Cumulat

maximum steps, simulation is done ... 
Episode: 182 Total reward: -184.53833093901787 Explore P: 0.0167 SOC: 0.6095 Cumulative_SOC_deviation: 13.5593 Fuel Consumption: 48.9455
maximum steps, simulation is done ... 
Episode: 183 Total reward: -357.72244305432673 Explore P: 0.0165 SOC: 0.8165 Cumulative_SOC_deviation: 29.2995 Fuel Consumption: 64.7274
maximum steps, simulation is done ... 
Episode: 184 Total reward: -192.18578692962814 Explore P: 0.0163 SOC: 0.6081 Cumulative_SOC_deviation: 14.3064 Fuel Consumption: 49.1215
maximum steps, simulation is done ... 
Episode: 185 Total reward: -232.43438804381205 Explore P: 0.0162 SOC: 0.7165 Cumulative_SOC_deviation: 17.4291 Fuel Consumption: 58.1439
maximum steps, simulation is done ... 
Episode: 186 Total reward: -224.34489934853318 Explore P: 0.0160 SOC: 0.6134 Cumulative_SOC_deviation: 17.4153 Fuel Consumption: 50.1918
maximum steps, simulation is done ... 
Episode: 187 Total reward: -378.9869938798374 Explore P: 0.0158 SOC: 0.8876 Cumul

In [11]:
with open("DDQN3_2_mass1200.pkl", "wb") as f: 
    pickle.dump(results_dict, f, pickle.HIGHEST_PROTOCOL)

In [12]:
# with open("results/replay_memory_size_effect.pkl", "rb") as f: 
#     data = pickle.load(f)
    
# data